### Shared Word

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# word sample
word_sample = True  # True, False
word_sample_num = 20

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [5]:
def remove_repetition(word_group):
    '''
    remove_repetition(word_group): detect word repetion in word group 
    '''
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331836,polygamisch,5
331837,shiit,5
331838,ambient,5
331839,zinswendingen,5


In [7]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [8]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
5,is,12813896
6,een,11124670
7,niet,10747590
8,en,8257870
9,van,6532887


In [9]:
#lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
#lang_pair_list

In [10]:
#df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,turkish_word
0,kanker,kanser
1,whisky,viski
2,liter,litre
3,gitaar,gitar
4,trein,tren
...,...,...
1563,zebra,zebra
1564,zigzag,zikzak
1565,zombie,zombi
1566,zoölogie,zooloji


In [11]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,turkish_word


In [12]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [13]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [14]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [15]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three", "Four", "Five", "Sentence"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}{file_ext}.xlsx", index=False)    

#### Concat Result

In [16]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,ik
1,in
2,maar
3,niet
4,zijn
5,wat
6,n
7,en
8,die
9,als


In [17]:
# Option For Frequency
if shared_word_frequency:
    df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
    df_word = pd.merge(df_word,df_word_all, how="inner", on="word")
    df_word.drop_duplicates(inplace=True)
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    pass

df_word

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
5,is,12813896
6,een,11124670
7,niet,10747590
8,en,8257870
9,van,6532887


In [18]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram{file_ext}.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,ik heb,1113451
1,het is,1102152
2,dat is,939846
3,wat is,698111
4,heb je,691326
...,...,...
746,te ze,4
747,de heb,3
748,te een,3
749,te n,3


In [19]:
# Option For Twogram Sentence
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram_sent = df_twogram_sent.loc[:,["twogram"]]
    df_shared_twogram = pd.merge(df_shared_twogram, df_twogram_sent, how="inner", on="twogram")
    df_shared_twogram.drop_duplicates(inplace=True)
    df_shared_twogram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_twogram


,twogram,freq_twogram
0,ik heb,1113451
1,het is,1102152
2,dat is,939846
3,wat is,698111
4,heb je,691326
...,...,...
746,te ze,4
747,de heb,3
748,te een,3
749,te n,3


In [20]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram{file_ext}.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,wat is er,364876
1,het is een,135216
2,ik heb een,110642
3,ik heb het,107020
4,wat is dat,92873
...,...,...
10028,in dat die,3
10029,is een is,3
10030,is een je,3
10031,in dat was,3


In [21]:
# Option For Threegram Sentence
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram_sent = df_twogram_sent.loc[:,["threegram"]]
    df_shared_threegram = pd.merge(df_shared_threegram, df_threegram_sent, how="inner", on="threegram")
    df_shared_threegram.drop_duplicates(inplace=True)
    df_shared_threegram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_threegram

,threegram,freq_threegram
0,wat is er,364876
1,het is een,135216
2,ik heb een,110642
3,ik heb het,107020
4,wat is dat,92873
...,...,...
10028,in dat die,3
10029,is een is,3
10030,is een je,3
10031,in dat was,3


In [22]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram{file_ext}.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram
0,wat is er met,43390
1,dat is het niet,14604
2,is er met je,12640
3,het is maar een,10266
4,dat heb ik niet,9969
...,...,...
20645,heb ik niet wat,3
20646,heb ik niet n,3
20647,heb er voor het,3
20648,heb er een ik,3


In [23]:
# Option For Fourgram Sentence
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fourgram_sent = df_twogram_sent.loc[:,["fourgram"]]
    df_shared_fourgram = pd.merge(df_shared_fourgram, df_fourgram_sent, how="inner", on="fourgram")
    df_shared_fourgram.drop_duplicates(inplace=True)
    df_shared_fourgram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_fourgram

,fourgram,freq_fourgram
0,wat is er met,43390
1,dat is het niet,14604
2,is er met je,12640
3,het is maar een,10266
4,dat heb ik niet,9969
...,...,...
20645,heb ik niet wat,3
20646,heb ik niet n,3
20647,heb er voor het,3
20648,heb er een ik,3


In [24]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram{file_ext}.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram
0,wat is er met je,12627
1,dat is niet wat ik,2548
2,het is niet wat je,2357
3,wat is er met de,1716
4,je dat ik dat niet,1503
...,...,...
10480,er is n ik en,3
10481,wat was dat in die,3
10482,wat was dat met een,3
10483,wat was dat met de,3


In [25]:
# Option For Fivegram Sentence
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fivegram_sent = df_twogram_sent.loc[:,["fivegram"]]
    df_shared_fivegram = pd.merge(df_shared_fivegram, df_fivegram_sent, how="inner", on="fivegram")
    df_shared_fivegram.drop_duplicates(inplace=True)
    df_shared_fivegram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_fivegram

,fivegram,freq_fivegram
0,wat is er met je,12627
1,dat is niet wat ik,2548
2,het is niet wat je,2357
3,wat is er met de,1716
4,je dat ik dat niet,1503
...,...,...
10480,er is n ik en,3
10481,wat was dat in die,3
10482,wat was dat met een,3
10483,wat was dat met de,3


In [26]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence{file_ext}.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,wat is er,106497
1,wat is dat,62768
2,wat was dat,20127
3,wat is het,18734
4,dat is het,17533
...,...,...
2002,wat als ze als,3
2003,en wat voor een is dat,3
2004,ik was een van ze,3
2005,niet er op,3


In [27]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,ik,22539531.0,ik heb,1113451.0,wat is er,364876.0,wat is er met,43390,wat is er met je,12627.0,wat is er,106497.0
1,je,20769946.0,het is,1102152.0,het is een,135216.0,dat is het niet,14604,dat is niet wat ik,2548.0,wat is dat,62768.0
2,het,15696161.0,dat is,939846.0,ik heb een,110642.0,is er met je,12640,het is niet wat je,2357.0,wat was dat,20127.0
3,de,15258816.0,wat is,698111.0,ik heb het,107020.0,het is maar een,10266,wat is er met de,1716.0,wat is het,18734.0
4,dat,13387137.0,heb je,691326.0,wat is dat,92873.0,dat heb ik niet,9969,je dat ik dat niet,1503.0,dat is het,17533.0
...,...,...,...,...,...,...,...,...,...,...,...,...
20645,NaN,NaN,NaN,NaN,NaN,NaN,heb ik niet wat,3,NaN,NaN,NaN,NaN
20646,NaN,NaN,NaN,NaN,NaN,NaN,heb ik niet n,3,NaN,NaN,NaN,NaN
20647,NaN,NaN,NaN,NaN,NaN,NaN,heb er voor het,3,NaN,NaN,NaN,NaN
20648,NaN,NaN,NaN,NaN,NaN,NaN,heb er een ik,3,NaN,NaN,NaN,NaN


In [28]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx", index=False)

##### Concat Result With Comma

In [29]:
def word_in_wordgroup(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            if word_sample:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)  # Option
            else:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [30]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_all, "word", "sentence") 

In [31]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()

In [32]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [33]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,als,"als je, als ik, als een, als we, als het, als ...","als je het, als je dat, als je me, als ik het,...","als je het niet, als ik dat niet, als je dat n...","en als ik dat niet, als dat is wat je, en als ...","als wat, wat als, als je, als ik, als we, het ..."
1,dat,"dat is, je dat, dat je, is dat, dat ik, dat wa...","wat is dat, dat is niet, dat is het, dat is ee...","dat is het niet, dat heb ik niet, wat is dat v...","dat is niet wat ik, je dat ik dat niet, wat is...","wat is dat, wat was dat, dat is het, dat is he..."
2,de,"in de, van de, is de, op de, voor de, met de, ...","ik heb de, dat is de, het is de, niet in de, h...","je me in de, je me voor de, me niet in de, hij...","wat is er met de, ik heb je in de, hij is een ...","de wat, wat de, en de, in de, de was, het is d..."
3,die,"je die, met die, is die, die is, van die, die ...","die heb ik, heb je die, ik heb die, die is er,...","die heb ik niet, die is er niet, is er met die...","wat is er met die, wat is dat met die, wat heb...","die is er niet, die heb ik niet, die heb ik, d..."
4,een,"is een, je een, heb een, was een, wat een, in ...","het is een, ik heb een, dat is een, hij is een...","het is maar een, het was maar een, ik heb er e...","wat is dat voor een, het is maar voor een, hij...","een wat, ik heb er een, het is een, dat is een..."
5,en,"en ik, en de, en dat, en je, en wat, en het, e...","en dat is, en ik heb, en het is, en wat is, en...","en wat is dat, en dat is het, en ik heb een, e...","en als ik dat niet, en wat is er met, en als z...","en dat is, en ik, en wat is dat, en wat, en hi..."
6,er,"is er, er is, er niet, je er, er zijn, zijn er...","wat is er, is er met, er is een, ik heb er, we...","wat is er met, is er met je, hij is er niet, w...","wat is er met je, wat is er met de, wat is er ...","wat is er, we zijn er, wat is er met je, hij i..."
7,heb,"ik heb, heb je, heb ik, heb een, wat heb, heb ...","ik heb een, ik heb het, ik heb je, wat heb je,...","dat heb ik niet, ik heb het niet, wat heb je m...","wat heb je voor me, ik heb wat voor je, wat he...","ik heb het, wat heb je, ik heb je, heb je het,..."
8,het,"het is, het niet, is het, je het, het was, dat...","het is een, ik heb het, dat is het, heb je het...","dat is het niet, het is maar een, ik heb het n...","het is niet wat je, je dat ik het niet, maar d...","wat is het, dat is het, ik heb het, het is, da..."
9,hij,"hij is, is hij, dat hij, hij was, hij niet, en...","hij is een, hij is niet, hij is er, hij is de,...","hij is er niet, hij is in de, dat is hij niet,...","hij is een van de, maar hij is er niet, en als...","hij is het, hij is er niet, hij is, hij is er,..."


In [34]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,ik,22539531,"ik heb, dat ik, ik niet, heb ik, wat ik, ik je...","ik heb een, ik heb het, ik heb je, dat heb ik,...","dat heb ik niet, ik heb het niet, dat is wat i...","dat is niet wat ik, je dat ik dat niet, ik heb...","ik niet, ik heb het, ik heb je, en ik, dat was..."
1,je,20769946,"heb je, je dat, dat je, je niet, je het, je me...","ik heb je, wat heb je, heb je het, je dat ik, ...","is er met je, is het met je, wat heb je met, i...","wat is er met je, het is niet wat je, je dat i...","wat heb je, ik heb je, wat is er met je, heb j..."
2,het,15696161,"het is, het niet, is het, je het, het was, dat...","het is een, ik heb het, dat is het, heb je het...","dat is het niet, het is maar een, ik heb het n...","het is niet wat je, je dat ik het niet, maar d...","wat is het, dat is het, ik heb het, het is, da..."
3,de,15258816,"in de, van de, is de, op de, voor de, met de, ...","ik heb de, dat is de, het is de, niet in de, h...","je me in de, je me voor de, me niet in de, hij...","wat is er met de, ik heb je in de, hij is een ...","de wat, wat de, en de, in de, de was, het is d..."
4,dat,13387137,"dat is, je dat, dat je, is dat, dat ik, dat wa...","wat is dat, dat is niet, dat is het, dat is ee...","dat is het niet, dat heb ik niet, wat is dat v...","dat is niet wat ik, je dat ik dat niet, wat is...","wat is dat, wat was dat, dat is het, dat is he..."
5,is,12813896,"het is, dat is, wat is, is er, is het, is een,...","wat is er, het is een, wat is dat, dat is niet...","wat is er met, dat is het niet, is er met je, ...","wat is er met je, dat is niet wat ik, het is n...","wat is er, wat is dat, wat is het, dat is het,..."
6,een,11124670,"is een, je een, heb een, was een, wat een, in ...","het is een, ik heb een, dat is een, hij is een...","het is maar een, het was maar een, ik heb er e...","wat is dat voor een, het is maar voor een, hij...","een wat, ik heb er een, het is een, dat is een..."
7,niet,10747590,"het niet, je niet, ik niet, is niet, me niet, ...","dat is niet, het is niet, is het niet, niet wa...","dat is het niet, dat heb ik niet, ik heb het n...","dat is niet wat ik, het is niet wat je, je dat...","ik niet, dat is het niet, wat niet, hij is er ..."
8,en,8257870,"en ik, en de, en dat, en je, en wat, en het, e...","en dat is, en ik heb, en het is, en wat is, en...","en wat is dat, en dat is het, en ik heb een, e...","en als ik dat niet, en wat is er met, en als z...","en dat is, en ik, en wat is dat, en wat, en hi..."
9,van,6532887,"van de, van je, van het, je van, van me, van e...","je van me, een van de, het is van, hij is van,...","dat ik van je, dat je van me, het is niet van,...","dat was ik niet van, hij is een van de, dat is...","van wat, het is van, ik was er n van, een van ..."


In [35]:
df_word_order_join_all.word.nunique()

28

In [36]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [37]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}.xlsx")
output_file

['Dutch_Turkish_Shared_Twogram6.xlsx',
 'Dutch_Turkish_Shared_Threegram6.xlsx',
 'Dutch_Turkish_Shared_Fourgram6.xlsx',
 'Dutch_Turkish_Shared_Fivegram6.xlsx',
 'Dutch_Turkish_Shared_Sentence6.xlsx',
 'Dutch_Turkish_Shared_Result_With_Frequency6.xlsx',
 'Dutch_Turkish_Shared_Join_Result_Without_Frequency6.xlsx']

In [38]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [39]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass